In [1]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [2]:
geisinger_df = pd.read_excel('GNSIS_DATABASE_v7.5_6.25.2020.xlsx')


In [3]:
data_master = pd.read_fwf('STROKE_STRICT_ALL_DXS.txt',header=None)

In [4]:
stroke_df = pd.read_excel('STROKE_STRICT_PROBLEM_LIST.xlsx', sheet_name='PROBLEM_LIST')
admission_df = pd.read_excel('STROKE_STRICT_PROBLEM_LIST.xlsx', sheet_name='ADMISION_ENCS')

In [5]:
data_master.columns = ['PT_ID','Dates','ENC_TYPE','ICD_CD','ICD_NM','CODE_SYSTEM','PRIMARY_YN'] 
data_master = data_master.iloc[2:]
data_master = data_master.iloc[:-3]
data_master[['INDEX_DT','ENC_DT']] = data_master['Dates'].str.split(expand=True) 
del data_master['Dates']
print(data_master['PT_ID'].nunique())

9243


In [6]:
stroke_df=stroke_df.rename(columns={'PROB_DT':'ENC_DT'})
stroke_df['ENC_TYPE'] = 'IP ONLY'
print(stroke_df['PT_ID'].nunique())
print(admission_df['PT_ID'].nunique())

8917
16


In [7]:
data_master = data_master[['PT_ID','INDEX_DT','ENC_DT','ENC_TYPE','ICD_CD','ICD_NM']]
admission_df = admission_df[['PT_ID','INDEX_DT','ENC_DT','ENC_TYPE','ICD_CD','ICD_NM']]

# merge data , stroke df and admission df into one dataframe

In [8]:
data = data_master.append(stroke_df, ignore_index = True,sort=True) 
data = data.append(admission_df, ignore_index = True,sort=True) 

In [9]:
print(data['PT_ID'].nunique())
df = pd.merge(geisinger_df['PT_ID'], data, on=['PT_ID'], how='inner')
print(df['PT_ID'].nunique())
print(geisinger_df['PT_ID'].nunique())

9243
8929
8929


In [13]:
df['PT_ID'] = df['PT_ID'].astype(str)
df['INDEX_DT'] = pd.to_datetime(df['INDEX_DT'])
df['ENC_DT'] = pd.to_datetime(df['ENC_DT'])
df.dtypes

PT_ID               object
ENC_DT      datetime64[ns]
ENC_TYPE            object
ICD_CD              object
ICD_NM              object
INDEX_DT    datetime64[ns]
dtype: object

In [14]:
df_afib_icd9 = df[df['ICD_CD'].isin (['427.31', '427.3'])]
df_afib_icd10 = df[df['ICD_CD'].isin(['I48','I48.0', 'I48.1','I48.11','I48.19', 'I48.2','I48.20', 'I48.21','I48.9','I48.91'])]
df_afib = pd.concat([df_afib_icd9, df_afib_icd10], ignore_index=True)

df_aflt_icd9 = df[df['ICD_CD'].isin (['427.32'])]
df_aflt_icd10 = df[df['ICD_CD'].isin(['I48.3', 'I48.4','I48.92'])]
df_aflt = pd.concat([df_aflt_icd9, df_aflt_icd10], ignore_index=True)

print(df_aflt['PT_ID'].nunique())
print(df_afib['PT_ID'].nunique())
df_afib_aflt = df_afib.append(df_aflt, ignore_index = True) 
print(df_afib_aflt['PT_ID'].nunique())

420
2793
2843


# Using rule of 2 (1 inpatient, 2 out patient contribute to 1)

In [21]:
df_afib_outPatient = df_afib[df_afib['ENC_TYPE'] == 'OUTPATIENT'].copy()
df_afib_inPatient = df_afib[df_afib['ENC_TYPE'] != 'OUTPATIENT']
print('AFIB Only')
print('outPatient: ',df_afib_outPatient['PT_ID'].nunique())
print('inPatient: ',df_afib_inPatient['PT_ID'].nunique())

print('===================================================')

df_afib_aflt_outPatient = df_afib_aflt[df_afib_aflt['ENC_TYPE'] == 'OUTPATIENT'].copy()
df_afib_aflt_inPatient = df_afib_aflt[df_afib_aflt['ENC_TYPE'] != 'OUTPATIENT']
print('AFIB + Aflutter')
print('outPatient: ',df_afib_aflt_outPatient['PT_ID'].nunique())
print('inPatient: ',df_afib_aflt_inPatient['PT_ID'].nunique())

df_afib_outPatient.dtypes

AFIB Only
outPatient:  1787
inPatient:  2722
AFIB + Aflutter
outPatient:  1837
inPatient:  2776


PT_ID               object
ENC_DT      datetime64[ns]
ENC_TYPE            object
ICD_CD              object
ICD_NM              object
INDEX_DT    datetime64[ns]
dtype: object

In [22]:
df_afib_outPatient = df_afib_outPatient.groupby('PT_ID')
df_afib_outPatient = df_afib_outPatient.filter(lambda x: len(x) > 1)
print(df_afib_outPatient['PT_ID'].nunique())
df_afib_outPatient=df_afib_outPatient.sort_values(by=['PT_ID','ENC_DT'])
testVal= df_afib_outPatient.groupby('PT_ID',as_index=False)['ENC_DT'].apply(lambda x: x.sort_values())
df_afib_outPatient['ENC_DT'] = testVal.reset_index(level=0, drop=True)
df_afib_outPatient = df_afib_outPatient.groupby('PT_ID').apply(lambda x: x.tail(len(x)-1))\
        .reset_index(drop=True)
print('===========================')

df_afib_aflt_outPatient = df_afib_aflt_outPatient.groupby('PT_ID')
df_afib_aflt_outPatient = df_afib_aflt_outPatient.filter(lambda x: len(x) > 1)
print(df_afib_aflt_outPatient['PT_ID'].nunique())

df_afib_aflt_outPatient=df_afib_aflt_outPatient.sort_values(by=['PT_ID','ENC_DT'])
val= df_afib_aflt_outPatient.groupby('PT_ID',as_index=False)['ENC_DT'].apply(lambda x: x.sort_values())
df_afib_aflt_outPatient['ENC_DT'] = val.reset_index(level=0, drop=True)
df_afib_aflt_outPatient = df_afib_aflt_outPatient.groupby('PT_ID').apply(lambda x: x.tail(len(x)-1))\
        .reset_index(drop=True)

1494
1545


In [23]:
diff_afib_op = df_afib_outPatient[~df_afib_outPatient['PT_ID'].isin(df_afib_inPatient['PT_ID'].unique())]
print(diff_afib_op['PT_ID'].nunique())

diff_afib_aflt_op = df_afib_aflt_outPatient[~df_afib_aflt_outPatient['PT_ID'].isin(df_afib_aflt_inPatient['PT_ID'].unique())]
print(diff_afib_aflt_op['PT_ID'].nunique())


22
22


In [24]:
data_afib_rule_2 = df_afib_inPatient.append(df_afib_outPatient, ignore_index = True,sort=True) 
print(data_afib_rule_2['PT_ID'].nunique())

2744


In [25]:
data_afib_aflt_rule_2 = df_afib_aflt_inPatient.append(df_afib_aflt_outPatient, ignore_index = True,sort=True) 
print(data_afib_aflt_rule_2['PT_ID'].nunique())

2798


# Testing with Geisinger Data

In [43]:
test_afib = geisinger_df[geisinger_df['ATRIAL_FIB_ALL_TIME'] == 1]
test_afib['PT_ID'].nunique()

2744

In [58]:
test_afib_aflt = geisinger_df[geisinger_df['AFIB_FLUTTER_ALL_TIME'] == 1]
test_afib_aflt['PT_ID'].nunique()

2798

In [59]:
diff_afib = test_afib[~test_afib['PT_ID'].isin(data_afib_rule_2['PT_ID'].unique())]
diff_afib_aflt = test_afib_aflt[~test_afib_aflt['PT_ID'].isin(data_afib_aflt_rule_2['PT_ID'].unique())]

In [60]:
print(diff_afib['PT_ID'].nunique())
print(diff_afib_aflt['PT_ID'].nunique())

0
0


# Prepare Afib data for later usage

In [72]:
data_afib_rule_2['INDEX_DT'] = pd.to_datetime(data_afib_rule_2['INDEX_DT'])
data_afib_rule_2['ENC_DT'] = pd.to_datetime(data_afib_rule_2['ENC_DT'])

data_afib_aflt_rule_2['INDEX_DT'] = pd.to_datetime(data_afib_aflt_rule_2['INDEX_DT'])
data_afib_aflt_rule_2['ENC_DT'] = pd.to_datetime(data_afib_aflt_rule_2['ENC_DT'])

In [27]:
import numpy as np
df_afib_data = data_afib_rule_2.copy(deep=True)
df_afib_data['Difference'] = (df_afib_data['ENC_DT'] - df_afib_data['INDEX_DT'])
df_afib_data['Difference_in_years'] = (df_afib_data['ENC_DT'] - df_afib_data['INDEX_DT'])/np.timedelta64(1,'Y')

df_afib_aflt_data = data_afib_aflt_rule_2.copy(deep=True)
df_afib_aflt_data['Difference'] = (df_afib_aflt_data['ENC_DT'] - df_afib_aflt_data['INDEX_DT'])
df_afib_aflt_data['Difference_in_years'] = (df_afib_aflt_data['ENC_DT'] - df_afib_aflt_data['INDEX_DT'])/np.timedelta64(1,'Y')

In [28]:
def set_year_wise_occurance(df):
   # print(cdf.iloc[:,k])
    df['Occurance_in_years'] = np.select(
        [
            (df['Difference_in_years'] >= -22) & (df['Difference_in_years'] < -21),
            (df['Difference_in_years'] >= -21) & (df['Difference_in_years'] < -20),
            (df['Difference_in_years'] >= -20) & (df['Difference_in_years'] < -19),
            (df['Difference_in_years'] >= -19) & (df['Difference_in_years'] < -18),
            (df['Difference_in_years'] >= -18) & (df['Difference_in_years'] < -17),
            (df['Difference_in_years'] >= -17) & (df['Difference_in_years'] < -16),
            (df['Difference_in_years'] >= -16) & (df['Difference_in_years'] < -15),
            (df['Difference_in_years'] >= -15) & (df['Difference_in_years'] < -14),
            (df['Difference_in_years'] >= -14) & (df['Difference_in_years'] < -13),
            (df['Difference_in_years'] >= -13) & (df['Difference_in_years'] < -12),
            (df['Difference_in_years'] >= -12) & (df['Difference_in_years'] < -11),
            (df['Difference_in_years'] >= -11) & (df['Difference_in_years'] < -10),
            (df['Difference_in_years'] >= -10) & (df['Difference_in_years'] < -9),
            (df['Difference_in_years'] >= -9) & (df['Difference_in_years'] < -8),
            (df['Difference_in_years'] >= -8) & (df['Difference_in_years'] < -7),
            (df['Difference_in_years'] >= -7) & (df['Difference_in_years'] < -6),
            (df['Difference_in_years'] >= -6) & (df['Difference_in_years'] < -5),
            (df['Difference_in_years'] >= -5) & (df['Difference_in_years'] < -4),
            (df['Difference_in_years'] >= -4) & (df['Difference_in_years'] < -3),
            (df['Difference_in_years'] >= -3) & (df['Difference_in_years'] < -2),
            (df['Difference_in_years'] >= -2) & (df['Difference_in_years'] < -1),
            (df['Difference_in_years'] >= -1) & (df['Difference_in_years'] < 0),
            df['Difference_in_years'] == 0,
            (df['Difference_in_years'] > 0) & (df['Difference_in_years'] <= 1),
            (df['Difference_in_years'] > 1) & (df['Difference_in_years'] <= 2), 
            (df['Difference_in_years'] > 2) & (df['Difference_in_years'] <= 3), 
            (df['Difference_in_years'] > 3) & (df['Difference_in_years'] <= 4), 
            (df['Difference_in_years'] > 4) & (df['Difference_in_years'] <= 5), 
            (df['Difference_in_years'] > 5) & (df['Difference_in_years'] <= 6), 
            (df['Difference_in_years'] > 6) & (df['Difference_in_years'] <= 7),
            (df['Difference_in_years'] > 7) & (df['Difference_in_years'] <= 8),
            (df['Difference_in_years'] > 8) & (df['Difference_in_years'] <= 9), 
            (df['Difference_in_years'] > 9) & (df['Difference_in_years'] <= 10), 
            (df['Difference_in_years'] > 10) & (df['Difference_in_years'] <= 11), 
            (df['Difference_in_years'] > 11) & (df['Difference_in_years'] <= 12), 
            (df['Difference_in_years'] > 12) & (df['Difference_in_years'] <= 13), 
            (df['Difference_in_years'] > 13) & (df['Difference_in_years'] <= 14), 
            (df['Difference_in_years'] > 14) & (df['Difference_in_years'] <= 15),
            (df['Difference_in_years'] > 15) & (df['Difference_in_years'] <= 16)
        ], 
        [
            -22,
            -21,
            -20,
            -19,
            -18,
            -17,
            -16,
            -15,
            -14,
            -13,
            -12,
            -11,
            -10,
            -9,
            -8,
            -7,
            -6,
            -5,
            -4,
            -3,
            -2,
            -1,
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16
        ], 
        default=0
    )
    return

In [29]:
set_year_wise_occurance(df_afib_data)
set_year_wise_occurance(df_afib_aflt_data)
print(df_afib_data['PT_ID'].nunique())
print(df_afib_aflt_data['PT_ID'].nunique())

2744
2798


## HISTOGRAM FOR PATIENTS USING FIRST ENCOUNTER OF AFIB BEFORE OR AFTER STROKE

In [46]:
#pat_PT1039775.sort_values(by=['Difference_in_years'])
df_afib_data = df_afib_data.sort_values(by=['PT_ID','Difference_in_years'])
df_afib_data = df_afib_data.drop_duplicates(subset=['PT_ID'])

df_afib_aflt_data = df_afib_aflt_data.sort_values(by=['PT_ID','Difference_in_years'])
df_afib_aflt_data = df_afib_aflt_data.drop_duplicates(subset=['PT_ID'])
df_afib_aflt_data = df_afib_aflt_data[df_afib_aflt_data['PT_ID']!= 'PT1078737'] 

df_afib_data.to_csv('df_afib_no_duplicate_per_patient.csv',index=False)
df_afib_aflt_data.to_csv('df_afib_aflt_no_duplicate_per_patient.csv',index=False)   

## Diagnosis By grouping

In [47]:
print(df_afib_data['PT_ID'].nunique())
df_afib_grouped =  df_afib_data.groupby(['ICD_CD','ICD_NM']) \
       .agg({'PT_ID':'nunique'}) \
       .reset_index()
df_afib_grouped

2744


,ICD_CD,ICD_NM,PT_ID
0,427.31,Atrial fibrillation,1802
1,I48.0,Paroxysmal atrial fibrillation,332
2,I48.1,Persistent atrial fibrillation,32
3,I48.2,Chronic atrial fibrillation,63
4,I48.91,Unspecified atrial fibrillation,515


In [48]:
print(df_afib_aflt_data['PT_ID'].nunique())
df_afib_aflt_grouped =  df_afib_aflt_data.groupby(['ICD_CD','ICD_NM']) \
       .agg({'PT_ID':'nunique'}) \
       .reset_index()
df_afib_aflt_grouped

2797


,ICD_CD,ICD_NM,PT_ID
0,427.31,Atrial fibrillation,1741
1,427.32,Atrial flutter,99
2,I48.0,Paroxysmal atrial fibrillation,323
3,I48.1,Persistent atrial fibrillation,31
4,I48.2,Chronic atrial fibrillation,61
5,I48.3,Typical atrial flutter,3
6,I48.4,Atypical atrial flutter,1
7,I48.91,Unspecified atrial fibrillation,498
8,I48.92,Unspecified atrial flutter,40


### Demographics

###### geisinger_afib_patients = geisinger_df[geisinger_df['PT_ID'].isin(df_afib_data['PT_ID'])]
data_full_afib_patients = df[df['PT_ID'].isin(df_afib_data['PT_ID'])]
data_full_afib_patients.to_csv('data_full_afib_patients.csv',index=False)
geisinger_afib_patients.to_csv('geisinger_afib_patients.csv',index=False)

geisinger_afib_aflt_patients = geisinger_df[geisinger_df['PT_ID'].isin(df_afib_aflt_data['PT_ID'])]
data_full_afib_aflt_patients = df[df['PT_ID'].isin(df_afib_aflt_data['PT_ID'])]
data_full_afib_aflt_patients.to_csv('data_full_afib_aflt_patients.csv',index=False)
geisinger_afib_aflt_patients.to_csv('geisinger_afib_aflt_patients.csv',index=False)

print(geisinger_afib_patients['PT_ID'].nunique())
print(geisinger_afib_aflt_patients['PT_ID'].nunique())

### Stroke Patients with Afib

In [51]:
geisinger_afib_patients_grouped = geisinger_afib_patients.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique', 'AGE_AT_INDEX':'mean'}) \
       .rename(columns={'PT_ID':'Distinct Patient','AGE_AT_INDEX':'Avg Age at index'}) \
       .reset_index()
print(geisinger_afib_patients['AGE_AT_INDEX'].mean())
geisinger_afib_patients_grouped

76.14482507288629


,PT_SEX,Distinct Patient,Avg Age at index
0,Female,1437,78.14
1,Male,1307,73.95


In [52]:
geisinger_afib_aflt_patients_grouped = geisinger_afib_aflt_patients.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique', 'AGE_AT_INDEX':'mean'}) \
       .rename(columns={'PT_ID':'Distinct Patient','AGE_AT_INDEX':'Avg Age at index'}) \
       .reset_index()
print(geisinger_afib_aflt_patients['AGE_AT_INDEX'].mean())
geisinger_afib_aflt_patients_grouped

76.07947801215589


,PT_SEX,Distinct Patient,Avg Age at index
0,Female,1468,78.07
1,Male,1329,73.88


### Stroke Patients without Afib

In [59]:
geisinger_non_afib_patients = geisinger_df[~geisinger_df['PT_ID'].isin(df_afib_data['PT_ID'])]
data_full_non_afib_patients = df[~df['PT_ID'].isin(df_afib_data['PT_ID'])]
geisinger_non_afib_patients.to_csv('geisinger_non_afib_patients.csv',index=False)
data_full_non_afib_patients.to_csv('data_full_non_afib_patients.csv',index=False)

geisinger_non_afib_aflt_patients = geisinger_df[~geisinger_df['PT_ID'].isin(df_afib_aflt_data['PT_ID'])]
data_full_non_afib_aflt_patients = df[~df['PT_ID'].isin(df_afib_aflt_data['PT_ID'])]
geisinger_non_afib_aflt_patients.to_csv('geisinger_non_afib_aflt_patients.csv',index=False)
data_full_non_afib_aflt_patients.to_csv('data_full_non_afib_aflt_patients.csv',index=False)

print(geisinger_non_afib_patients['PT_ID'].nunique())
print(geisinger_non_afib_aflt_patients['PT_ID'].nunique())

6185
6132


In [54]:
geisinger_non_afib_patients_grouped = geisinger_non_afib_patients.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique', 'AGE_AT_INDEX':'mean'}) \
       .rename(columns={'PT_ID':'Distinct Patient','AGE_AT_INDEX':'Avg Age at index'}) \
       .reset_index()
print(geisinger_non_afib_patients['AGE_AT_INDEX'].mean())
geisinger_non_afib_patients_grouped

67.29902991107518


,PT_SEX,Distinct Patient,Avg Age at index
0,Female,2882,69.17
1,Male,3303,65.67


In [55]:
geisinger_non_afib_aflt_patients_grouped = geisinger_non_afib_aflt_patients.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique', 'AGE_AT_INDEX':'mean'}) \
       .rename(columns={'PT_ID':'Distinct Patient','AGE_AT_INDEX':'Avg Age at index'}) \
       .reset_index()
print(geisinger_non_afib_aflt_patients['AGE_AT_INDEX'].mean())
geisinger_non_afib_aflt_patients_grouped

67.25238095238095


,PT_SEX,Distinct Patient,Avg Age at index
0,Female,2851,69.11
1,Male,3281,65.64


In [57]:
Afib_cohort_geisinger_before_index = geisinger_df[geisinger_df['ATRIAL_FIB_PREINDEX'] == 1]
Afib_Aflut_cohort_geisinger_before_index = geisinger_df[geisinger_df['AFIB_FLUTTER_PREINDEX'] == 1]

data_afib_diagnosed_before_storke = df_afib_data[df_afib_data['Occurance_in_years'] < 0]
data_afib_aflt_diagnosed_before_storke = df_afib_aflt_data[df_afib_aflt_data['Occurance_in_years'] < 0]

print('===================Before Stroke index========================')
print('Afib Geisinger: ',Afib_cohort_geisinger_before_index['PT_ID'].nunique())
print('Afib + Aflut Geisinger: ',Afib_Aflut_cohort_geisinger_before_index['PT_ID'].nunique())

print('Afib Master: ',data_afib_diagnosed_before_storke['PT_ID'].nunique())
print('Afib + Aflut Master: ',data_afib_aflt_diagnosed_before_storke['PT_ID'].nunique())

remainings = data_afib_diagnosed_before_storke[~data_afib_diagnosed_before_storke['PT_ID'].isin(Afib_cohort_geisinger_before_index['PT_ID'])]
print('Afib_data remainings: ',remainings['PT_ID'].nunique())

remainings = data_afib_aflt_diagnosed_before_storke[~data_afib_aflt_diagnosed_before_storke['PT_ID'].isin(Afib_Aflut_cohort_geisinger_before_index['PT_ID'])]
print('Afib_aflt_data remainings: ',remainings['PT_ID'].nunique())

print('===================================================')

Afib_cohort_geisinger_at_index = geisinger_df[(geisinger_df['ATRIAL_FIB_PREINDEX'] == 0) & (geisinger_df['ATRIAL_FIB_AT_INDEX'] == 1)]
Afib_Aflut_cohort_geisinger_at_index = geisinger_df[(geisinger_df['AFIB_FLUTTER_PREINDEX'] == 0) & (geisinger_df['AFIB_FLUTTER_AT_INDEX'] == 1)]

data_afib_diagnosed_at_storke = df_afib_data[df_afib_data['Occurance_in_years'] == 0]
data_afib_aflt_diagnosed_at_storke = df_afib_aflt_data[df_afib_aflt_data['Occurance_in_years'] == 0]

print('===================At Stroke index========================')
print('Afib Geisinger: ',Afib_cohort_geisinger_at_index['PT_ID'].nunique())
print('Afib + Aflut Geisinger: ',Afib_Aflut_cohort_geisinger_at_index['PT_ID'].nunique())

print('Afib Master: ',data_afib_diagnosed_at_storke['PT_ID'].nunique())
print('Afib + Aflut Master: ',data_afib_aflt_diagnosed_at_storke['PT_ID'].nunique())

remainings = data_afib_diagnosed_at_storke[~data_afib_diagnosed_at_storke['PT_ID'].isin(Afib_cohort_geisinger_at_index['PT_ID'])]
print('Afib_data remainings: ',remainings['PT_ID'].nunique())

remainings = data_afib_aflt_diagnosed_at_storke[~data_afib_aflt_diagnosed_at_storke['PT_ID'].isin(Afib_Aflut_cohort_geisinger_at_index['PT_ID'])]
print('Afib_aflt_data remainings: ',remainings['PT_ID'].nunique())
print('===================================================')

Afib_cohort_geisinger_after_index = geisinger_df[(geisinger_df['ATRIAL_FIB_PREINDEX'] == 0) & (geisinger_df['ATRIAL_FIB_AT_INDEX'] == 0)  & (geisinger_df['ATRIAL_FIB_ALL_TIME'] == 1)]
Afib_Aflut_cohort_geisinger_after_index = geisinger_df[(geisinger_df['AFIB_FLUTTER_PREINDEX'] == 0) & (geisinger_df['AFIB_FLUTTER_AT_INDEX'] == 0)  & (geisinger_df['AFIB_FLUTTER_ALL_TIME'] == 1)]
data_afib_diagnosed_after_storke = df_afib_data[df_afib_data['Occurance_in_years'] > 0]
data_afib_aflt_diagnosed_after_storke = df_afib_aflt_data[df_afib_aflt_data['Occurance_in_years'] > 0]

print('===================After Stroke index========================')
print('Afib Geisinger: ',Afib_cohort_geisinger_after_index['PT_ID'].nunique())
print('Afib + Aflut Geisinger: ',Afib_Aflut_cohort_geisinger_after_index['PT_ID'].nunique())

print('Afib Master: ',data_afib_diagnosed_after_storke['PT_ID'].nunique())
print('Afib + Aflut Master: ',data_afib_aflt_diagnosed_after_storke['PT_ID'].nunique())    
                                                                                                                    
remainings = data_afib_diagnosed_after_storke[~data_afib_diagnosed_after_storke['PT_ID'].isin(Afib_cohort_geisinger_after_index['PT_ID'])]
print('Afib_data remainings: ',remainings['PT_ID'].nunique())

remainings = data_afib_aflt_diagnosed_after_storke[~data_afib_aflt_diagnosed_after_storke['PT_ID'].isin(data_afib_aflt_diagnosed_after_storke['PT_ID'])]
print('Afib_aflt_data remainings: ',remainings['PT_ID'].nunique())
                                                                                                                    
print('===================================================')


===================Before Stroke index========================
Afib Geisinger:  1062
Afib + Aflut Geisinger:  1090
Afib Master:  1062
Afib + Aflut Master:  1090
Afib_data remainings:  0
Afib_aflt_data remainings:  0
===================At Stroke index========================
Afib Geisinger:  819
Afib + Aflut Geisinger:  842
Afib Master:  819
Afib + Aflut Master:  841
Afib_data remainings:  0
Afib_aflt_data remainings:  0
===================After Stroke index========================
Afib Geisinger:  863
Afib + Aflut Geisinger:  866
Afib Master:  863
Afib + Aflut Master:  866
Afib_data remainings:  0
Afib_aflt_data remainings:  0


In [60]:
print(data_full_non_afib_patients['PT_ID'].nunique())
print(data_full_non_afib_aflt_patients['PT_ID'].nunique())

6185
6132


In [61]:
data_afib_aflt_rule_2[data_afib_aflt_rule_2['PT_ID']!= 'PT1078737'] 

,ENC_DT,ENC_TYPE,ICD_CD,ICD_NM,INDEX_DT,PT_ID
0,2008-11-20,ED TO IP,427.31,Atrial fibrillation,2008-11-20,PT1039670
1,2008-11-24,IP ONLY,427.31,Atrial fibrillation,2008-11-20,PT1039670
2,2011-10-25,ED TO IP,427.31,Atrial fibrillation,2012-07-31,PT1039775
3,2011-09-02,ED TO IP,427.31,Atrial fibrillation,2012-07-31,PT1039775
4,2007-03-30,IP ONLY,427.31,Atrial fibrillation,2012-07-31,PT1039775
...,...,...,...,...,...,...
34275,2011-10-31,OUTPATIENT,427.31,Atrial fibrillation,2004-01-26,PT999898
34276,2012-01-31,OUTPATIENT,427.31,Atrial fibrillation,2004-01-26,PT999898
34277,2012-02-28,OUTPATIENT,427.31,Atrial fibrillation,2004-01-26,PT999898
34278,2012-06-26,OUTPATIENT,427.31,Atrial fibrillation,2004-01-26,PT999898
